In [1]:
% setup environment
% clear data and number format
clear; format short e
% set path to equilibrium solver
txt=pwd; n=length(txt); rootpath=txt;
n=strfind(rootpath,"USERS"); rootpath=txt(1:n-2);
PHREEQCpath=[rootpath,'/runPHREEQC'];
addpath(PHREEQCpath);
JEQUILIBpath=[rootpath,'/JEQUILIB'];
addpath(JEQUILIBpath);
% turn off warnings
warning off
%graphics_toolkit ("notebook");
graphics_toolkit ("plotly");
%graphics_toolkit("gnuplot")
%delete existing phreeqc files
system("rm DATABASE.dat");
system("rm runfile.txt");
%create empty phreeqc files
system("touch DATABASE.dat");
system("touch runfile.txt");

In [2]:
% define equilibrium problem

%FeT=1e-5; PT=1e-4; NaT=3*PT; ClT=3*FeT; T=25; HFOsi=1e-30; HFOwi=1e-30; ASFw=1; ASFs=1;
%pH=2:0.2:11;
FeT=1e-4/0.997; PT=1e-6/0.997; NaT=3*PT; ClT=3*FeT; T=25; HFOsi=1e-30; HFOwi=1e-30; ASFw=0.2; ASFs=0.005; %/0.997 to mmol/kg at 25C
pH=2:0.2:12;


% set redox level ------------------------
peINT=20.75; %oxic
%peINT=15.75; %less oxic
%peINT=10.75; %getting reducing
%peINT=5.75; %pretty reducing
%peINT=0; % maximum reducing

pe=peINT-pH;% oxic end

% ──────────── pre-alocar resultados ────────────
nPts        = numel(pH);
Fep3        = zeros(1,nPts);   % Fe3+ acuoso
HFO         = zeros(1,nPts);   % Fe(OH)3(s)
HFO_P_ads   = zeros(1,nPts);   % PO4 adsorbido total (fuerte+débil)
FePO4s      = zeros(1,nPts);   % Fe(PO4)(s)  (strengite)
Fe3PO42s    = zeros(1,nPts);   % Fe3(PO4)2(s) (vivianita)
MASSERR     = zeros(1,nPts);   % error de masa %
ErrComp_store = cell(1, nPts); % Un cell array para guardar los nombres


In [ ]:
% === MODELO USANDO PHREEQC (flag0 = 1) =========================
flag0 = 1;   flag1 = 2;  flag2 = 1;  flag3 = 0;  flag4 = 1;  flag5 = 0;

TOTALS = [FeT; PT; ClT; NaT; HFOsi; HFOwi]';
TOTALS(TOTALS==0) = 1e-16;          % evitar div/0

% --- INICIO DE MEDICIÓN DE TIEMPO ---
tic; % Inicia el cronómetro para el tiempo de reloj (wall time)
t_cpu_inicio = cputime; % Guarda el tiempo de CPU inicial

for i = 1:numel(pH)
    % ---- primera corrida (sin “site” actualizado) ----
    [Fep3(i), HFO(i), HFO_P_ads(i), ...
     FePO4s(i), Fe3PO42s(i), MASSERR(i), ErrComp_store{i}] = ...
        FeHPO4tableau_CS(pH(i), pe(i), TOTALS', ...
                         flag0,flag1,flag2,flag3,flag4,flag5);

    % ---- si precipitó HFO, volver a resolver con sitios escalados ----
    if HFO(i) > 0
        HFOs   = ASFs * HFO(i);
        HFOw   = ASFw * HFO(i);
        TOTALS = [FeT; PT; ClT; NaT; HFOs; HFOw]';

        [Fep3(i), HFO(i), HFO_P_ads(i), ...
         FePO4s(i), Fe3PO42s(i), MASSERR(i), ErrComp_store{i}] = ...
            FeHPO4tableau_CS(pH(i), pe(i), TOTALS', ...
                             flag0,flag1,flag2,flag3,flag4,flag5);
    end

    %disp([pH(i) Fep3(i) HFO(i) HFO_P_ads(i) FePO4s(i) Fe3PO42s(i) MASSERR(i)])
end

% --- FIN DE MEDICIÓN Y REPORTE ---
tiempo_wall = toc; % Detiene el cronómetro y obtiene el tiempo de reloj transcurrido
tiempo_cpu = cputime - t_cpu_inicio; % Calcula el tiempo de CPU transcurrido

% Muestra los resultados en la consola
fprintf('\n--- Tiempos de Ejecución del Bucle ---\n');
fprintf('Tiempo de reloj (wall time): %.4f segundos\n', tiempo_wall);
fprintf('Tiempo de CPU:               %.4f segundos\n', tiempo_cpu);


In [ ]:
plot(pH, Fep3,  ...
     pH, HFO,  ...
     pH, HFO_P_ads, ...
     pH, FePO4s, ...
     pH, Fe3PO42s)

xlabel('pH');
ylabel('[concentración mol kg^{-1}_{w}]');
legend({'Fe^{3+}_{(aq)}', ...
        'Fe(OH)_3(s)', ...
        'PO_4 ads', ...
        'FePO_4(s)', ...
        'Fe_3(PO_4)_2(s)'}, ...
        'location','eastoutside');


In [ ]:
% --- CELDA 5: GRÁFICA DE ERROR ---
figure;
plot(pH, MASSERR, '-o', 'LineWidth', 1.5);
xlabel('pH');
ylabel('Error Máximo de Balance de Masa');
title('Error de la Simulación vs. pH');
grid on;

In [ ]:
quickRun = struct( ...
    "pH",           pH(:),        ...
    "Fe3_aq",       Fep3(:),      ...
    "FeOH3s",       HFO(:),       ...
    "HFO_P_ads",    HFO_P_ads(:), ...
    "FePO4s",       FePO4s(:),    ...
    "Fe3PO42s",     Fe3PO42s(:),  ...
    "MassErr",      abs(MASSERR(:)),...    
    "TimeWall_s",  tiempo_wall,  ... 
    "TimeCPU_s",   tiempo_cpu    ... 
);

save("-v7", "phreeqc_runsCS.mat", "quickRun");
printf("\nquickRun guardado con los datos de la sesión rápida.\n");


In [7]:
ErrComp_store

ErrComp_store = {
  [1,1] = Fe
  [1,2] = Fe
  [1,3] = Fe
  [1,4] = Fe
  [1,5] = Fe
  [1,6] = Fe
  [1,7] = Fe
  [1,8] = Fe
  [1,9] = Fe
  [1,10] = Fe
  [1,11] = Fe
  [1,12] = Fe
  [1,13] = Fe
  [1,14] = Fe
  [1,15] = Fe
  [1,16] = Fe
  [1,17] = Fe
  [1,18] = Fe
  [1,19] = Fe
  [1,20] = Fe
  [1,21] = Fe
  [1,22] = Fe
  [1,23] = Fe
  [1,24] = Fe
  [1,25] = Fe
  [1,26] = Fe
  [1,27] = Fe
  [1,28] = Fe
  [1,29] = Fe
  [1,30] = Fe
  [1,31] = Fe
  [1,32] = Fe
  [1,33] = Fe
  [1,34] = Fe
  [1,35] = Fe
  [1,36] = Fe
  [1,37] = Fe
  [1,38] = Fe
  [1,39] = Fe
  [1,40] = Fe
  [1,41] = Fe
  [1,42] = Fe
  [1,43] = Fe
  [1,44] = Fe
  [1,45] = Fe
  [1,46] = Fe
  [1,47] = Fe
  [1,48] = Fe
  [1,49] = Hfow
  [1,50] = Fe
  [1,51] = Fe
}


In [8]:
pH

pH =  Columns 1 through 5:

    2.0000e+00    2.2000e+00    2.4000e+00    2.6000e+00    2.8000e+00

 Columns 6 through 10:

    3.0000e+00    3.2000e+00    3.4000e+00    3.6000e+00    3.8000e+00

 Columns 11 through 15:

    4.0000e+00    4.2000e+00    4.4000e+00    4.6000e+00    4.8000e+00

 Columns 16 through 20:

    5.0000e+00    5.2000e+00    5.4000e+00    5.6000e+00    5.8000e+00

 Columns 21 through 25:

    6.0000e+00    6.2000e+00    6.4000e+00    6.6000e+00    6.8000e+00

 Columns 26 through 30:

    7.0000e+00    7.2000e+00    7.4000e+00    7.6000e+00    7.8000e+00

 Columns 31 through 35:

    8.0000e+00    8.2000e+00    8.4000e+00    8.6000e+00    8.8000e+00

 Columns 36 through 40:

    9.0000e+00    9.2000e+00    9.4000e+00    9.6000e+00    9.8000e+00

 Columns 41 through 45:

    1.0000e+01    1.0200e+01    1.0400e+01    1.0600e+01    1.0800e+01

 Columns 46 through 50:

    1.1000e+01    1.1200e+01    1.1400e+01    1.1600e+01    1.1800e+01

 Column 51:

    1.2000e+01
